# compare TKE dissipation rate with rain

In [ ]:
# preamble
using Revise
using Pkg; Pkg.activate(".")

using Dates
using Statistics
using Rotations
using Interpolations
using DSP
using FFTW
using NCDatasets
using JLD2
using Printf

include("./read_lidar.jl")
using .read_lidar
using .read_lidar.stare
using .read_vecnav: read_vecnav_dict
import .chunks
read_stare_time  = Main.chunks.read_stare_time
read_stare_chunk = Main.chunks.read_stare_chunk
fit_offset = Main.chunks.fit_offset

include("./timing_lidar.jl")
using .timing_lidar
include("./readers.jl")
using .NoaaDas: cat_dicts
# using MAT

using PyPlot
using PyCall
using PyCall: PyObject

# PyObject method interprets Array{Union{T,Missing}} as a
# numpy masked array.
# This allows for plotting with missing values.
function PyObject(a::Array{Union{T,Missing},N}) where {T,N}
    numpy_ma = PyCall.pyimport("numpy").ma
    pycall(numpy_ma.array, Any, coalesce.(a,zero(T)), mask=ismissing.(a))
end

In [ ]:
# helper functions 
pd = permutedims
m2n(x) = ismissing(x) ? NaN : x

In [ ]:
# load TKE dissipation rate data
ds_e = load("epsilon_data/epsi_stare_chunks_r20240604_1533.jld", "epsi", "dtime_st", "dtime_en", "height")
epsi     = ds_e["epsi"]
dtime_st = ds_e["dtime_st"]
dtime_en = ds_e["dtime_en"]
height   = ds_e["height"]

# load PSL met data
# candidate files in order of preference
files = [ "./data/../PSL/netcdf/EUREC4A_ATOMIC_RonBrown_10min_nav_met_sea_flux_20200109-20200212_v1.3.nc",
          "/Users/deszoeks/Projects/ASTRAL/SSTcompare/data/PSL/ASTRAL-nav-met-sea-flux-10min_RV-Thompson_20240428_R1_thru_20240613.nc" ]
filename = files[findfirst(isfile, files)]
psl = NCDataset(filename)

In [ ]:
# plot

clf()
subplot(2,1,1)
pcolormesh(start_dt, height/1e3, log10.(plotfilt.(pd(epsi))), vmin = -6, vmax = -1.5,
        cmap=ColorMap("RdYlBu_r"))
pcolormesh(psl[:time], height[1:4]/1e3, log10.(plotfilt.(pd(epsilon0[:,1:4]))), vmin = -6, vmax = -1.5,
        cmap=ColorMap("RdYlBu_r"))
gca().xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%m%d"))
title("EKAMSAT atmospheric BL TKE dissipation rate")
# plot(psl[:time][:], 0.5*psl[:prate][:], linewidth=0.5, color="blue")
# plot(psl[:time][:], 0.1*psl[:wspd_10][:], linewidth=0.5, color="k")
# plot(psl[:time][:], 0.1*psl[:rspd][:], linewidth=0.5, color="k")
# # plot(psl[:time][:], 0.75 .+ 0.75/180*psl[:rdir][:], linewidth=0.5, color="m")
# plot(psl[:time][:], 1000*bflx0, linewidth=0.5, color="r")
xlim([DateTime(2024,4,29), DateTime(2024,5,14)])
# xlim([DateTime(2024,5,2), DateTime(2024,5,4)])
ylim([0, 1.5])
gca().set_facecolor(0.6.+[0,0,0])
gca().xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%m.%d"))

subplot(2,1,2)
pcolormesh(psl[:time], height[1:4]/1e3, log10.(plotfilt.(pd(epsilon0[:,1:4]))), vmin = -6, vmax = -1.5,
        cmap=ColorMap("RdYlBu_r"))
pcolormesh(start_dt, height/1e3, log10.(plotfilt.(pd(epsi))), vmin = -6, vmax = -1.5,
        cmap=ColorMap("RdYlBu_r"))
# plot(psl[:time][:], 0.5*psl[:prate][:], linewidth=0.5, color="blue")
# plot(psl[:time][:], 0.1*psl[:wspd_10][:], linewidth=0.5, color="k")
# plot(psl[:time][:], 0.1*psl[:rspd][:], linewidth=0.5, color="k")
# # plot(psl[:time][:], 0.75 .+ 0.75/180*psl[:rdir][:], linewidth=0.5, color="m")
# plot(psl[:time][:], 1000*bflx0, linewidth=0.5, color="r")
xlim([DateTime(2024,6,1), DateTime(2024,6,11)])
# xlim([DateTime(2024,6,2), DateTime(2024,6,4)])
ylim([0, 1.5])
gca().set_facecolor(0.6.+[0,0,0])
gca().xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%m.%d"))
cb = colorbar(); cb.set_label("log"*L"_{10}\epsilon")
tight_layout()
gcf()

In [ ]:
# interpolate psl to lidar time
"convert datetime to Integer milliseconds, which interpolate can handle."
dt2ms(dt, dt0=DateTime(2024,4,1)) = Dates.value(Millisecond(dt - dt0))
"interpolate from psl datetime to lidar dt"
interp_psl_to_lidar(y; psldt=psl[:time][:], lidardt=start_dt) = 
    interpolate( (dt2ms.(psldt), ), y, Gridded(Linear()) )(dt2ms.(lidardt)) # returns a vector